In [1]:
import csv
import re
import sys
import time
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import metrics, tree
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_selection import chi2, SelectKBest

In [2]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str, encoding = 'utf-8',
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])
#df = shuffle(df)

In [3]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [4]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
#     punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
#     no_punct = ""
#     for char in sample:
#         if char not in punctuations:
#             no_punct = no_punct + char
#     return no_punct
    return re.sub(r'[^\w\s\&\#\@\$\%\_]','',sample)

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au') and not bool(re.search(r'\d',word))]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = [w for w in sample.split(' ') if len(w) >= 2]
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = [w for w in sample.split(' ') if len(w) >= 2]
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()

def lemmy(sample):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    words = [w for w in sample.split(' ') if len(w) >= 2]
    lemmed_text = ""
    for word in words:
        lemmed_text = lemmed_text + lemmatizer.lemmatize(word) + " "
    return lemmed_text.rstrip()
    
def snowball(sample):
    words = [w for w in sample.split(' ') if len(w) >= 2]
    stemmer = SnowballStemmer("english")
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + stemmer.stem(word) + " "
    return stemmed_text.rstrip()

def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
#     sample = lemmy(sample)
#     sample = snowball(sample)
    sample = porter_stem(sample)
    return sample


In [5]:
#myTokenizer('While the majority of all Internet search engines utilize stop words, they do not prevent a user from using them, but they are ignored.')

In [28]:
# try to use sklearn stop_words later
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=1200, ngram_range=(1, 2))
# count = TfidfVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=800,sublinear_tf=True, min_df=5,
#                         ngram_range=(1, 2), 
#                         stop_words='english')
bag_of_words = count.fit_transform(text_data)
# print(count.get_feature_names())
# size = len(count.vocabulary_)
print(len(count.vocabulary_))

1200


In [29]:
X = bag_of_words.toarray()
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)
# First 1500 for training set, last 500 for test set
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)

In [30]:
clf = MultinomialNB(alpha = 1)
model = clf.fit(X_train, y_train)
#model = clf.fit(X, Y)
print(model.class_log_prior_ )
text_clf_red = Pipeline([('vect', CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer)), 
                       ('reducer', SelectKBest(chi2, k=200)),
                       ('clf', MultinomialNB())
                       ])
model_new = text_clf_red.fit(text_data[:1500],Y[:1500])
# model_new = text_clf_red.fit(text_data,Y)

[-2.07677842 -2.66882949 -2.71810054 -1.71110157 -4.60517019 -2.35739333
 -2.33648664 -5.70378247 -2.55104645 -4.82831374 -3.5065579  -5.52146092
 -4.26869795 -3.10852777 -4.17772617 -2.7593435  -3.5065579  -3.75787233
 -3.98101588 -3.70230247]


In [31]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10000       0.59      0.71      0.65        56
       10001       0.36      0.25      0.30        36
       10002       0.58      0.48      0.53        31
       10003       0.33      0.53      0.41        87
       10004       0.00      0.00      0.00         2
       10005       0.59      0.62      0.60        52
       10006       0.42      0.41      0.41        44
       10007       0.00      0.00      0.00         2
       10008       0.56      0.63      0.59        46
       10009       0.00      0.00      0.00         4
       10010       0.14      0.09      0.11        11
       10011       0.00      0.00      0.00         7
       10012       0.00      0.00      0.00         4
       10013       0.69      0.49      0.57        37
       10014       0.67      0.33      0.44         6
       10015       0.54      0.62      0.58        24
       10016       0.17      0.07      0.10        14
       10017       0.00    

C:\Users\King\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

       10000       0.83      0.86      0.84       188
       10001       0.80      0.67      0.73       104
       10002       0.71      0.81      0.76        99
       10003       0.69      0.76      0.73       271
       10004       1.00      0.47      0.64        15
       10005       0.74      0.83      0.78       142
       10006       0.80      0.83      0.81       145
       10007       1.00      0.20      0.33         5
       10008       0.79      0.86      0.82       117
       10009       0.75      0.50      0.60        12
       10010       0.56      0.62      0.59        45
       10011       0.00      0.00      0.00         6
       10012       0.69      0.43      0.53        21
       10013       0.85      0.90      0.87        67
       10014       1.00      0.48      0.65        23
       10015       0.83      0.96      0.89        95
       10016       0.78      0.47      0.58        45
       10017       0.95    

C:\Users\King\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
y_pred = model_new.predict(text_data[1500:])
print(classification_report(Y[1500:], y_pred))

              precision    recall  f1-score   support

       10000       0.71      0.36      0.48        56
       10001       0.53      0.22      0.31        36
       10002       0.56      0.61      0.58        31
       10003       0.24      0.71      0.36        87
       10004       0.00      0.00      0.00         2
       10005       0.79      0.65      0.72        52
       10006       0.57      0.18      0.28        44
       10007       0.00      0.00      0.00         2
       10008       0.70      0.72      0.71        46
       10009       0.00      0.00      0.00         4
       10010       0.75      0.55      0.63        11
       10011       0.00      0.00      0.00         7
       10012       0.00      0.00      0.00         4
       10013       0.83      0.41      0.55        37
       10014       1.00      0.33      0.50         6
       10015       0.62      0.62      0.62        24
       10016       0.50      0.14      0.22        14
       10017       0.00    

In [12]:
y_pred = model_new.predict(text_data[:1500])
print(classification_report(Y[:1500], y_pred))

              precision    recall  f1-score   support

       10000       0.71      0.45      0.55       188
       10001       0.66      0.28      0.39       104
       10002       0.60      0.73      0.66        99
       10003       0.32      0.84      0.47       271
       10004       1.00      0.53      0.70        15
       10005       0.62      0.56      0.59       142
       10006       0.74      0.22      0.34       145
       10007       1.00      0.80      0.89         5
       10008       0.76      0.66      0.71       117
       10009       0.60      0.25      0.35        12
       10010       0.80      0.27      0.40        45
       10011       1.00      0.50      0.67         6
       10012       0.83      0.48      0.61        21
       10013       0.80      0.52      0.63        67
       10014       1.00      0.22      0.36        23
       10015       0.84      0.89      0.87        95
       10016       0.67      0.13      0.22        45
       10017       1.00    